In [1]:
import pandas as pd
import sys
import glob
import os

from tqdm import tqdm

sys.path.append("../../../../utils")
from absolute_path_builder import AbsolutePathBuilder

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [3]:
DATASET = "coraal_vs_buckeye"

input_path = AbsolutePathBuilder.get_path(
    f"06_{DATASET}_regression",
    filepaths="../../../../config/filepaths.yaml"
)

In [4]:
df = None
for SCORE_MODEL in tqdm(
    [
        "perspective",
        "flair",
        "textblob",
        "vader",
        "detoxify_original",
        "detoxify_unbiased",
        "detoxify_multilingual"
    ]
):
    df_model = pd.read_csv(os.path.join(input_path, f"{SCORE_MODEL}_score_coefs.csv"))
    
    # Select the most relevant columns according to the p-value and the coefficient
    df_model = (
        df_model[df_model.pvalue < 0.01]
        .sort_values(
            "coef",
            key=lambda item: abs(item),
            ascending=False
        ).iloc[:25]
        [["column", "coef"]]
    )

    df_model = df_model.rename(columns={"coef": f"{SCORE_MODEL}"})
    
    if df is None:
        df = df_model
    else:
        df = df.merge(df_model, on="column", how="outer")

100%|██████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 408.77it/s]


In [5]:
df.dropna(thresh=5).reset_index(drop=True)

,column,perspective,flair,textblob,vader,detoxify_original,detoxify_unbiased,detoxify_multilingual
0,LIWC_SWEAR,3.1290,0.6116,NaN,0.8671,4.0893,4.1557,4.1065
1,LIWC_SEXUAL,0.8789,1.3414,0.1556,NaN,1.1509,0.8967,0.9980
2,LIWC_DEATH,0.7062,0.8989,-0.2076,1.1810,0.7096,0.6390,0.4368
3,LIWC_BODY,0.6777,1.6425,NaN,0.1656,0.8246,0.4927,0.6237
4,LIWC_ANGER,0.6152,0.5529,0.0499,0.2712,0.7251,0.8413,0.8249
5,LIWC_NEGEMO,0.4572,1.0030,0.8668,0.7982,0.2994,0.3219,0.3944
6,AAE_TERMS_COUNT,0.3095,0.4181,NaN,NaN,0.2153,0.2050,0.1806
7,LIWC_SAD,-0.2303,NaN,-0.4479,0.1435,-0.1457,-0.1755,-0.2405
8,LIWC_SEE,0.1854,NaN,NaN,NaN,0.2014,0.2549,0.1971
9,LIWC_SOCIAL,0.1407,NaN,NaN,NaN,0.1635,0.0542,0.0592
